# BLOCK-T231 - TMA Azimuth Brake Distance

In [ ]:
# %load_ext lab_black
import numpy as np
import os

from lsst.ts.observing import ObservingBlock, ObservingScript

In [ ]:
name = "BLOCK-T231"
program = "BLOCK-T231"
reason = "SITCOM-1536"
constraints = []
scripts = []

output_folder = "output_blocks"

In [ ]:
def build_configuration_schema(block_number, properties):
    """
    Builds a configuration schema string for a given BLOCK and configurable
    properties.

    Parameters
    ----------
    block_number :
        The BLOCK number to include in the title and description.
    properties : dict
        A dictionary where each key is a property name, and each value is a
        dictionary with keys 'description', 'type', and optionally 'default'.

    Returns
    -------
        A formatted configuration schema string.
    """

    # Define the base schema with the BLOCK number
    configuration_schema = (
        "$schema: http://json-schema.org/draft-07/schema#\n"
        f"title: BLOCK-{block_number} configuration\n"
        f"description: Configuration for BLOCK-{block_number}.\n"
        "type: object\n"
        "properties:\n"
    )

    # Add each property to the schema
    for prop_name, prop_details in properties.items():
        configuration_schema += f"  {prop_name}:\n"
        configuration_schema += f'    description: {prop_details["description"]}\n'
        configuration_schema += f'    type: {prop_details["type"]}\n'
        if "default" in prop_details:
            # Add quotes around the default value if it's a string
            default_value = prop_details["default"]
            if prop_details["type"] == "string":
                default_value = f'"{default_value}"'
            configuration_schema += f"    default: {default_value}\n"

    return configuration_schema

Define the configurable properties that we will use in the configuration schema

In [ ]:
properties = {
    "azimuth": {
        "description": "Target Azimuth position",
        "type": "number",
        "default": "0",
    },
    "elevation": {
        "description": "Target Elevation position",
        "type": "number",
        "default": "80",
    },
    "ignore": {
        "description": "Name of the CSCs we want to ignore",
        "type": "array",
        "default": [],
    },
    "move_timeout": {
        "description": "Timeout associated with the move_p2p scripts",
        "type": "number",
        "default": 1800,
    },
}

block_number = name.split("-")[-1]
configuration_schema = build_configuration_schema(block_number, properties)
print(configuration_schema)

In [ ]:
move_p2p = ObservingScript(
    name="maintel/move_p2p.py",
    standard=True,
    parameters=dict(
        az="$azimuth", el="$elevation", ignore="$ignore", move_timeout="$move_timeout"
    ),
)

scripts.append(move_p2p)

In [ ]:
block = ObservingBlock(
    name=name,
    program=program,
    configuration_schema=configuration_schema,
    scripts=scripts,
)

In [ ]:
block.model_dump_json(indent=2)

os.makedirs(output_folder, exist_ok=True)
output_path = f"{output_folder}/{name}.json"

with open(output_path, "w") as file:
    file.write(block.model_dump_json(indent=2))